### Mecab 설치

In [1]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.


In [2]:
cd Mecab-ko-for-Google-Colab/

/content/Mecab-ko-for-Google-Colab


In [3]:
! bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 13.6MB/s 
     |████████████████████████████████| 460kB 55.2MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-02-14 16:12:44--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c3:9b0a, 2406:da00:ff00::22e9:9f55, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downlo

### import 및 setting

In [4]:
from keras.preprocessing import sequence
from keras.models import load_model

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import adjusted_rand_score

from collections import defaultdict
import pickle, json, os
import re

from konlpy.tag import Mecab
mecab = Mecab()

# model load
model = load_model('/content/drive/MyDrive/STT_/model/Naver_dict_l3stm.h5')

# tokenizer load
with open("/content/drive/MyDrive/STT_/tokenizer/tokenizer3.pickle", 'rb') as fp:
    tokenizer = pickle.load(fp)

# token load
with open("/content/drive/MyDrive/STT_/token/token3.json", "r") as st_json:
    topic = json.load(st_json)

# stop words load
path ='/content/drive/MyDrive/STT_/korean_stopwords.txt'
f = open(path, 'r', encoding='utf-8')
stop_words = []
while True:
    line = f.readline()
    if not line: break
    stop_words.append(line.split()[0])
f.close()

max_word =20000
max_len = 1000 #max_len = 500

In [68]:
## 받아올 것 -> API 부분임
str_ = u'''
교활(狡猾)과 낭패(狼狽)는 상상의 동물 이름이다. 
이 교활이란 놈은 어찌나 간사한지 여우를 능가할 정도인데, 중국의 기서(奇書)인 『산해경(山海經)』에 등장하는 동물이다. 
교(狡)라는 놈은 모양은 개인데 온몸에 표범의 무늬가 있으며, 머리에는 쇠뿔을 달고 있다고 한다. 
이놈이 나타나면 그해에는 대풍(大豊)이 든다고 하는데, 이 녀석이 워낙 간사하여 나올 듯 말 듯 애만 태우다가 끝내 나타나지 않는다고 한다. 
한편 이 교의 친구로 활(猾)이라는 놈이 있는데 이놈은 교보다 더 간악하다. 
이놈은 생김새는 사람 같은데 온몸에 돼지털이 숭숭 나 있으며 동굴 속에 살면서 겨울잠을 잔다. 
도끼로 나무를 찍는 듯한 소리를 내는데, 이놈이 나타나면 온 천하가 대란(大亂)에 빠진다고 한다.
이처럼 교와 활은 간악하기로 유명한 동물인데, 길을 가다가 호랑이라도 만나면 몸을 똘똘 뭉쳐 조그만 공처럼 변신하여 제 발로 호랑이 입속으로 뛰어들어 내장을 마구 파먹는다. 
호랑이가 그 아픔을 참지 못해 뒹굴다가 죽으면 그제야 유유히 걸어나와 미소를 짓는다. 
여기에서 바로 그 ‘교활한 미소’라는 관용구가 생겨났다.

메탄올(methanol 또는 methyl alcohol) 또는 메틸 알코올이라고 부르며 나무로부터 많이 얻어진다고 하여 목정(wood spirit) 이라고도 한다. 
1661년에 보일(Robert Boyle)에 의해 회양목의 증류 과정에서 최초로 분리되었다. 
MeOH로 쓰기도 하며, 알코올 중에서 가장 간단한 구조로 되어 있다. 
물보다 가볍고, 무색의 가연성이 있는 극성을 띠는 액체이며, 맛과 냄새는 술의 주성분인 에탄올과 비슷하다. 
부동액, 연료 등으로 쓰이며 다양한 화학 반응에서 용매 또는 주요한 출발 물질로 사용된다.
'''

nums = 2


In [69]:
topic

{'0': '곤충',
 '1': '행정_기관_단체',
 '10': '만화',
 '11': '사회복지',
 '12': '음악용어',
 '13': '직업',
 '14': '아프리카',
 '15': '독일문학',
 '16': '무역',
 '17': '건축물',
 '18': '육아',
 '19': '북아메리카',
 '2': '한국_철학_윤리',
 '20': '건설_토목',
 '21': '농학',
 '22': '프랑스문학',
 '23': '영미문학',
 '24': '중동_역사_문화',
 '25': '한국_민속',
 '26': '무기',
 '27': '속담',
 '28': '기계_자동차_금속',
 '29': '국제기구',
 '3': '기타종교',
 '30': '외교',
 '31': '이슬람교',
 '32': '패션',
 '33': '경제',
 '34': '댄스_무용',
 '35': '동물',
 '36': '주류',
 '37': '한국_역사_문화',
 '38': '생명과학',
 '39': '호텔',
 '4': '커피_차',
 '40': '법',
 '41': '북한문학',
 '42': '아시아',
 '43': '남아메리카',
 '44': '헤어',
 '45': '서양_철학_윤리',
 '46': '불교',
 '47': '정치',
 '48': '재료',
 '49': '교양영어사전',
 '5': '클래식',
 '50': '명화',
 '51': '심리',
 '52': '화학',
 '53': '수학',
 '54': '물리학',
 '55': '세계_역사_문화',
 '56': '세계음식',
 '57': '중국문학',
 '58': '미술',
 '59': '한국음식',
 '6': '영화',
 '60': '의학',
 '61': '환경',
 '62': '색채',
 '63': '드라마',
 '64': '한국문학',
 '65': '아시아_역사_문화',
 '66': '교양_인문',
 '67': '질병',
 '68': '천문학',
 '69': '뮤지컬',
 '7': '군사',
 '70': '식물',


### PreProcessing

In [70]:
str_ = re.sub('[^A-Za-z0-9가-힣.]', ' ', str_)
str_ = re.sub(' +', ' ', str_)

In [71]:
documents = str_.split('.')
new_docs = []
for i in documents[:-1]:
  temp = ''
  for j in mecab.nouns(i):
    temp += j + ' '
  new_docs.append(temp)
new_docs

['교활 과 낭 상상 동물 이름 ',
 '교활 놈 여우 능가 정도 중국 기서 산해경 등장 동물 ',
 '교 놈 모양 개 온몸 표범 무늬 머리 쇠뿔 ',
 '이놈 그해 대풍 녀석 간사 듯 듯 애 ',
 '교의 친구 활 놈 이놈 교 간악 ',
 '이놈 생김새 사람 온몸 돼지 털 동굴 속 겨울잠 ',
 '도끼 나무 듯 소리 이놈 천하 대란 ',
 '이 교와 활 간악 동물 길 호랑이 몸 공 변신 제 발 호랑이 입속 내장 ',
 '호랑이 아픔 미소 ',
 '여기 미소 관용구 ',
 '메탄올 메틸 알코올 나무 목정 ',
 '년 회양목 증류 과정 최초 분리 ',
 '알코올 중 구조 ',
 '물 무색 가연 극성 액체 맛 냄새 술 주성분 에탄올 ',
 '부동액 연료 등 화학 반응 용매 출발 물질 사용 ']

### k-means clustering -> 문단 나누기

In [80]:
paragraphs = []
paragraphs2 = []
paragraphs3 = []
paragraphs4 = []

In [81]:
# tfidf -> 단어 vector화
vectorizer = TfidfVectorizer(max_features = max_word)
sp_matrix = vectorizer.fit_transform(new_docs)
sp_matrix

<15x69 sparse matrix of type '<class 'numpy.float64'>'
	with 81 stored elements in Compressed Sparse Row format>

In [82]:
Kmodel = KMeans(n_clusters=nums, init='k-means++', max_iter=800, n_init=4)
Kmodel.fit(sp_matrix)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=800,
       n_clusters=2, n_init=4, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [83]:
order_centroids = Kmodel.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [84]:
for i in range(nums):
  paragraphs.append([x for x, y in zip(documents, Kmodel.labels_) if  y == i])
  paragraphs2.append([x for x, y in zip(new_docs, Kmodel.labels_) if y == i])

In [85]:
paragraphs2

[['교활 과 낭 상상 동물 이름 ',
  '교활 놈 여우 능가 정도 중국 기서 산해경 등장 동물 ',
  '교 놈 모양 개 온몸 표범 무늬 머리 쇠뿔 ',
  '이놈 그해 대풍 녀석 간사 듯 듯 애 ',
  '교의 친구 활 놈 이놈 교 간악 ',
  '이놈 생김새 사람 온몸 돼지 털 동굴 속 겨울잠 ',
  '도끼 나무 듯 소리 이놈 천하 대란 ',
  '이 교와 활 간악 동물 길 호랑이 몸 공 변신 제 발 호랑이 입속 내장 ',
  '메탄올 메틸 알코올 나무 목정 ',
  '년 회양목 증류 과정 최초 분리 ',
  '알코올 중 구조 ',
  '물 무색 가연 극성 액체 맛 냄새 술 주성분 에탄올 ',
  '부동액 연료 등 화학 반응 용매 출발 물질 사용 '],
 ['호랑이 아픔 미소 ', '여기 미소 관용구 ']]

In [86]:
for i in range(len(paragraphs2)):
  temp,temp2 = '', ''
  for j in paragraphs[i]:
    temp += j
  for j in paragraphs2[i]:
    temp2 += j
  paragraphs3.append(temp)
  paragraphs4.append(temp2.split()[:-1])

In [87]:
del paragraphs2

### keyword 추출

In [90]:
def rmmult(sort_):
  lili = []
  for i in range(len(sort_)):
      if (sort_[i] in lili) or (sort_[i][1] < 0.01):
          continue
      else:
          lili.append(sort_[i])
  return lili

def rmmult_noarr(sort_):
    lili = []
    for i in range(len(sort_)):
        if (sort_[i][0] in lili) or (sort_[i][1] == 0):
            continue
        else:
            lili.append(sort_[i][0])
    return lili

In [91]:
tfidf_res_li = []
for p in paragraphs:
  vectorizer = TfidfVectorizer()
  sp_matrix = vectorizer.fit_transform(p)

  word2id = defaultdict(lambda : 0)
  for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

  li = []

  for i, sentence in enumerate(p):
    morph = mecab.nouns(sentence)
    a = [(token, sp_matrix[i, word2id[token]]) for token in morph]
    sortt = sorted(a, key=lambda k:k[1])
    sortt = rmmult(sortt)

    li += [(token, sp_matrix[i, word2id[token]]) for token in morph]
    
    sort_li = sorted(li, key=lambda k: k[1])
  a = sort_li[::-1]

  lili = rmmult_noarr(a)
  good_lili = []

  for j in lili:
    if j not in stop_words:
      good_lili.append(j)
    else: continue
  tfidf_res_li.append(good_lili[:5])



In [92]:
tfidf_res_li

[['동물', '교활', '교의', '알코올', '분리'], ['미소', '아픔', '호랑이']]

### model predcit

In [93]:
subject_classifier = []
topic_idx = []
for i in range(nums):
  sequences = tokenizer.texts_to_sequences([paragraphs4[i]])
  sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)
  topic_idx.append(model.predict_classes(sequences_matrix))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [94]:
topic_idx

[array([37]), array([57])]

### model 결과 및 전달 할 것 3개

In [95]:
for i in range(nums):
  print("----------------paragraph-----------------")
  print(paragraphs[i])

  print("----------------subject_classifier-----------------")
  print(topic[str(int(topic_idx[i]))])

  print("----------------keyword-----------------")
  print(tfidf_res_li[i])

  print()
  print()
  

----------------paragraph-----------------
[' 교활 과 낭패 는 상상의 동물 이름이다', ' 이 교활이란 놈은 어찌나 간사한지 여우를 능가할 정도인데 중국의 기서 인 산해경 에 등장하는 동물이다', ' 교 라는 놈은 모양은 개인데 온몸에 표범의 무늬가 있으며 머리에는 쇠뿔을 달고 있다고 한다', ' 이놈이 나타나면 그해에는 대풍 이 든다고 하는데 이 녀석이 워낙 간사하여 나올 듯 말 듯 애만 태우다가 끝내 나타나지 않는다고 한다', ' 한편 이 교의 친구로 활 이라는 놈이 있는데 이놈은 교보다 더 간악하다', ' 이놈은 생김새는 사람 같은데 온몸에 돼지털이 숭숭 나 있으며 동굴 속에 살면서 겨울잠을 잔다', ' 도끼로 나무를 찍는 듯한 소리를 내는데 이놈이 나타나면 온 천하가 대란 에 빠진다고 한다', ' 이처럼 교와 활은 간악하기로 유명한 동물인데 길을 가다가 호랑이라도 만나면 몸을 똘똘 뭉쳐 조그만 공처럼 변신하여 제 발로 호랑이 입속으로 뛰어들어 내장을 마구 파먹는다', ' 메탄올 methanol 또는 methyl alcohol 또는 메틸 알코올이라고 부르며 나무로부터 많이 얻어진다고 하여 목정 wood spirit 이라고도 한다', ' 1661년에 보일 Robert Boyle 에 의해 회양목의 증류 과정에서 최초로 분리되었다', ' MeOH로 쓰기도 하며 알코올 중에서 가장 간단한 구조로 되어 있다', ' 물보다 가볍고 무색의 가연성이 있는 극성을 띠는 액체이며 맛과 냄새는 술의 주성분인 에탄올과 비슷하다', ' 부동액 연료 등으로 쓰이며 다양한 화학 반응에서 용매 또는 주요한 출발 물질로 사용된다']
----------------subject_classifier-----------------
한국_역사_문화
----------------keyword-----------------
['동물', '교활', '교의', '알코올', '분리']


----------------paragraph-----------------
